In [198]:
import requests
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 1. Node > Django

In [6]:
node_origin = 'http://localhost:3000'
url = node_origin + '/test/nodedb_to_django'+'/420777'

In [7]:
res = requests.get(url)
print(res)

<Response [200]>


In [10]:
res.json()

[{'ingredient_name': '코다리'},
 {'ingredient_name': '고추'},
 {'ingredient_name': '고추가루'},
 {'ingredient_name': '간장'},
 {'ingredient_name': '설탕'},
 {'ingredient_name': '파'},
 {'ingredient_name': '마늘'},
 {'ingredient_name': '생강'},
 {'ingredient_name': '후추'},
 {'ingredient_name': '물엿'},
 {'ingredient_name': '소주'},
 {'ingredient_name': '참기름'},
 {'ingredient_name': '깨'},
 {'ingredient_name': '올리브유'},
 {'ingredient_name': '물'}]

In [185]:
retaining

,ingredient
0,코다리 고추 고추가루 간장 설탕 파 마늘 생강 후추 물엿 소주 참기름 깨 올리브유 물


# 2. Django db에 넣을 데이터 format

In [27]:
ori = pd.read_json('django_db.json')
ori

,recipe_info_id,ingredient_name,view
0,420777,코다리,252299
1,420777,고추,252299
2,667367,부추,116104
3,667367,양파,116104
4,667367,당근,116104
...,...,...,...
2390,5840021,낙지,114278
2391,5840021,대파,114278
2392,5840021,애호박,114278
2393,5840021,소금,114278


In [137]:
df = ori[['recipe_info_id','view']].drop_duplicates(keep='first')
df

,recipe_info_id,view
0,420777,252299
2,667367,116104
9,682697,327176
11,690101,171511
18,693694,110973
...,...,...
2377,5816092,236561
2381,5817718,134708
2384,5821585,230254
2390,5840021,114278


In [142]:
df['ingredient'] = ''

In [143]:
for r, c in ori.iterrows():
    corresponding_id = df['recipe_info_id'] == c[0]
    df.loc[corresponding_id, 'ingredient'] += ' '+c[1]
    # print(df.loc[corresponding_id, 'ingredient'])
    

In [145]:
df.to_json('django_db_wan.json')

In [156]:
df.head(3)

,recipe_info_id,view,ingredient
0,420777,252299,코다리 고추
2,667367,116104,부추 양파 당근 고추 가루 가루 얼음
9,682697,327176,돼지고기 고추


# 3. vector로 변환

In [ ]:
node_origin = 'http://localhost:3000'
url = node_origin + '/test/nodedb_to_django'+'/420777'
res = requests.get(url)

# 갖고 있는 재료를 django db 형식에 맞게 반환
col = 'ingredient'
retaining_ing = []
for r in res.json():
    retaining_ing.append(r['ingredient_name'])
retaining = pd.DataFrame(data={col:[' '.join(current_ing)]})

In [195]:
# 기존 데이터에 임시로 추가

tmp = df[['ingredient']].append(retaining, ignore_index=True)

In [157]:
# 벡터 변환기 생성
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
count_vect = count_vect.fit(df['ingredient'])

# 벡터 변환 > 유사도 계산 > 정렬
ing_mat = count_vect.transform(tmp['ingredient'])       # (497,995)
ing_sim = cosine_similarity(ing_mat, ing_mat)
ing_sim_sorted_ind = ing_sim.argsort()[:,::-1]

# 유사한 순으로 추출 > 조회수 많은 순으로 정렬
top_n = 5
similar_indexes = ing_sim_sorted_ind[tmp.shape[0]-1, 1:2*top_n]
df.iloc[similar_indexes].sort_values('view', ascending=False)[:5]